In [2]:
import numpy as np
import hdf5storage
import pyfftw
import tensorflow as tf
from IPython.display import Audio
from models.generator_sp import Model_Simple
from models.model_based_beamforming_v3 import model_based_beamforming

# load multi-channel noisy speech and the corresponding noise
s = hdf5storage.loadmat('assets/ch_8_8')['s_']
noise = hdf5storage.loadmat('assets/nch_8_8')['noise_']   

In [3]:
# build wavenet based enhancement model
len_pad = 4093
len_input = 48000
tf.reset_default_graph()
with tf.device('/gpu:0'):
    model = Model_Simple(len_input, len_input-2*len_pad)

model.saver.restore(model.sess, 'assets/bawn_sp.ckpt-399125')

Instructions for updating:
`NCHW` for data_format is deprecated, use `NCW` instead
INFO:tensorflow:Restoring parameters from assets/bawn_sp.ckpt-399125


In [4]:
# iteratively enhance and beamform
frame_len = s.shape[0]
h_order = 400
num_iters = 4

# return beamformed signal and beamformed noise
s_beamformed, noise_beamformed, _ = model_based_beamforming(model, 
                                                            s, 
                                                            noise, 
                                                            frame_len, 
                                                            len_input, 
                                                            h_order, 
                                                            num_iters)

Frame 0
Iteration 0 completed
Iteration 1 completed
Iteration 2 completed


In [5]:
Audio(s_beamformed[:,0], rate=16000)

In [6]:
Audio(noise_beamformed[:,0], rate=16000)

In [7]:
# compute signal to noise ratio
snr = 20*np.log10(np.linalg.norm(s_beamformed-noise_beamformed)\
                              /np.linalg.norm(noise_beamformed))
print('SNR = {}'.format(snr)) 

SNR = 32.7700721677
